<a href="https://colab.research.google.com/github/ishank296/Data-Analysis-Using-Pyspark/blob/main/002_pyspark_ckbk_perpare_date_for_modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- **Handling duplicates**
- **Handling missing observations**
- **Handling outliers**
- **Exploring descriptive statistics**
- **Computing correlations**
- **Drawing histograms**
- **Visualizing interactions between feature**

1. Installing spark over colab and create SparkSession/SparkContext

---






In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [5]:
!wget -q https://dlcdn.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz

In [7]:
!tar xf spark-3.3.0-bin-hadoop3.tgz

In [9]:
!pip install -q findspark

In [11]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"

In [22]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [14]:
spark

In [16]:
sc=spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

2. Create DataFrame from Sample data

In [17]:
dirty_data = spark.createDataFrame([
 (1,'Porsche','Boxster S','Turbo',2.5,4,22,None)
 , (2,'Aston Martin','Vanquish','Aspirated',6.0,12,16,None)
 , (3,'Porsche','911 Carrera 4S Cabriolet','Turbo',3.0,6,24,None)
 , (3,'General Motors','SPARK ACTIV','Aspirated',1.4,None,32,None)
 , (5,'BMW','COOPER S HARDTOP 2 DOOR','Turbo',2.0,4,26,None)
 , (6,'BMW','330i','Turbo',2.0,None,27,None)
 , (7,'BMW','440i Coupe','Turbo',3.0,6,23,None)
 , (8,'BMW','440i Coupe','Turbo',3.0,6,23,None)
 , (9,'Mercedes-Benz',None,None,None,None,27,None)
 , (10,'Mercedes-Benz','CLS 550','Turbo',4.7,8,21,79231)
 , (11,'Volkswagen','GTI','Turbo',2.0,4,None,None)
 , (12,'Ford Motor Company','FUSION AWD','Turbo',2.7,6,20,None)
 , (13,'Nissan','Q50 AWD RED SPORT','Turbo',3.0,6,22,None)
 , (14,'Nissan','Q70 AWD','Aspirated',5.6,8,18,None)
 , (15,'Kia','Stinger RWD','Turbo',2.0,4,25,None)
 , (16,'Toyota','CAMRY HYBRID LE','Aspirated',2.5,4,46,None)
 , (16,'Toyota','CAMRY HYBRID LE','Aspirated',2.5,4,46,None)
 , (18,'FCA US LLC','300','Aspirated',3.6,6,23,None)
 , (19,'Hyundai','G80 AWD','Turbo',3.3,6,20,None)
 , (20,'Hyundai','G80 AWD','Turbo',3.3,6,20,None)
, (21,'BMW','X5 M','Turbo',4.4,8,18,121231)
 , (22,'GE','K1500 SUBURBAN 4WD','Aspirated',5.3,8,18,None)
 ], ['Id','Manufacturer','Model','EngineType','Displacement',
 'Cylinders','FuelEconomy','MSRP'])

In [18]:
dirty_data.printSchema()

root
 |-- Id: long (nullable = true)
 |-- Manufacturer: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- EngineType: string (nullable = true)
 |-- Displacement: double (nullable = true)
 |-- Cylinders: long (nullable = true)
 |-- FuelEconomy: long (nullable = true)
 |-- MSRP: long (nullable = true)



## **Handling duplicates**

1. Check whether any rows are duplicated, as follows:

In [19]:
dirty_data.count(),dirty_data.distinct().count()

(22, 21)

2. Remove Duplicates

In [20]:
full_removed = dirty_data.dropDuplicates()
full_removed.count()

21

In [21]:
dirty_data.groupby(dirty_data.columns)\
          .count()\
          .filter('count>1')\
          .show()

+---+------------+---------------+----------+------------+---------+-----------+----+-----+
| Id|Manufacturer|          Model|EngineType|Displacement|Cylinders|FuelEconomy|MSRP|count|
+---+------------+---------------+----------+------------+---------+-----------+----+-----+
| 16|      Toyota|CAMRY HYBRID LE| Aspirated|         2.5|        4|         46|null|    2|
+---+------------+---------------+----------+------------+---------+-----------+----+-----+



3. Duplicates rows with different IDs

In [24]:
full_removed.groupBy([column for column in dirty_data.columns if column!='Id'])\
          .count()\
          .filter('count>1')\
          .show()

+------------+----------+----------+------------+---------+-----------+----+-----+
|Manufacturer|     Model|EngineType|Displacement|Cylinders|FuelEconomy|MSRP|count|
+------------+----------+----------+------------+---------+-----------+----+-----+
|         BMW|440i Coupe|     Turbo|         3.0|        6|         23|null|    2|
|     Hyundai|   G80 AWD|     Turbo|         3.3|        6|         20|null|    2|
+------------+----------+----------+------------+---------+-----------+----+-----+



In [25]:
no_ids = full_removed.drop('Id')
no_ids.distinct().count(),full_removed.count()

(19, 21)

In [26]:
id_removed = full_removed.dropDuplicates([col for col in full_removed.columns if col!='Id'])
id_removed.count()

19

4. ID Collisions

In [28]:
from pyspark.sql.functions import count,countDistinct
id_removed.agg(
    count('Id').alias('countOfIds'),
    countDistinct('Id').alias('countOfDistinctIds')
).show()

+----------+------------------+
|countOfIds|countOfDistinctIds|
+----------+------------------+
|        19|                18|
+----------+------------------+

